In [1]:
import numpy as np 
import pandas as pd
import os

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Flatten, Dense, LSTM, Dropout, Embedding, Activation
from keras.layers import concatenate, BatchNormalization, Input
from keras.layers import add
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.utils import plot_model

import matplotlib.pyplot as plt
import cv2
import string
import time

In [3]:
token_path = r"C:\Users\Aditya\Desktop\Fl_30_Dataset\captions.txt"
text = open(token_path, 'r', encoding = 'utf-8').read()
print(text[:500])

1000092795.jpg, Two young guys with shaggy hair look at their hands while hanging out in the yard .
1000092795.jpg," Two young , White males are outside near many bushes ."
1000092795.jpg, Two men in green shirts are standing in a yard .
1000092795.jpg, A man in a blue shirt standing in a garden .
1000092795.jpg, Two friends enjoy time spent together .
10002456.jpg, Several men in hard hats are operating a giant pulley system .
10002456.jpg, Workers look down from up above on a piece of equipmen


In [4]:
def load_description(text):
    mapping = dict()
    for line in text.split("\n"):
        if len(line) < 2:
            continue
        
        # Split the line based on the first occurrence of a comma
        img_id_with_extension, img_des = line.split(",", 1)
        
        # Remove the file extension
        img_id_without_extension = img_id_with_extension.split('.')[0].strip()
        
        img_des = img_des.strip()  # Remove leading/trailing whitespaces
        
        if img_id_without_extension not in mapping:
            mapping[img_id_without_extension] = list()
        mapping[img_id_without_extension].append(img_des)
    
    return mapping

# Assuming 'text' contains your captions
descriptions = load_description(text)

# To print descriptions for the specific image '10002456'
image_id_without_extension = '10002456'
if image_id_without_extension in descriptions:
    print(f"\nDescriptions for image {image_id_without_extension}:")
    for desc in descriptions[image_id_without_extension]:
        print(desc)
else:
    print(f"\nNo descriptions found for image {image_id_without_extension}")


Descriptions for image 10002456:
Several men in hard hats are operating a giant pulley system .
Workers look down from up above on a piece of equipment .
Two men working on a machine wearing hard hats .
Four men on top of a tall structure .
Three men on a large rig .


In [5]:
# Print all image IDs available in 'descriptions'
print("All image IDs in descriptions:")
for img_id in descriptions.keys():
    print(img_id)

# To print descriptions for the specific image '10002456'
image_id = '10002456.jpg'
if image_id in descriptions:
    print(f"\nDescriptions for image {image_id}:")
    for desc in descriptions[image_id]:
        print(desc)
else:
    print(f"\nNo descriptions found for image {image_id}")


All image IDs in descriptions:
1000092795
10002456
1000268201
1000344755
1000366164
1000523639
1000919630
10010052
1001465944
1001545525
1001573224
1001633352
1001773457
1001896054
100197432
100207720
1002674143
1003163366
1003420127
1003428081
100444898
1005216151
100577935
1006452823
100652400
1007129816
100716317
1007205537
1007320043
100759042
10082347
10082348
100845130
10090841
1009434119
1009692167
101001624
1010031975
1010087179
1010087623
10101477
1010470346
1010673430
101093029
101093045
1011572216
1012150929
1012212859
1012328893
101262930
1013536888
101362133
101362650
1014609273
101471792
1014785440
1015118661
1015584366
101559400
1015712668
10160966
101654506
1016626169
101669240
1016887272
1017675163
1018057225
1018148011
101859883
10188041
1019077836
101958970
1019604187
102030371
1020651753
1021293940
1021332107
1021439420
1021442086
1022454332
1022454428
102268204
1022975728
102351840
1024138940
102455176
1024613706
102520526
102575576
102617084
1026685415
1026792563


In [6]:
def clean_description(desc):
    for key, des_list in desc.items():
        for i in range(len(des_list)):
            caption = des_list[i]
            caption = [ch for ch in caption if ch not in string.punctuation]
            caption = ''.join(caption)
            caption = caption.split(' ')
            caption = [word.lower() for word in caption if len(word)>1 and word.isalpha()]
            caption = ' '.join(caption)
            des_list[i] = caption

clean_description(descriptions)
descriptions['10002456']

['several men in hard hats are operating giant pulley system',
 'workers look down from up above on piece of equipment',
 'two men working on machine wearing hard hats',
 'four men on top of tall structure',
 'three men on large rig']

In [7]:
def to_vocab(desc):
    words = set()
    for key in desc.keys():
        for line in desc[key]:
            words.update(line.split())
    return words
vocab = to_vocab(descriptions)
len(vocab)

19735

In [8]:
import glob

# Update the file path to the directory where your images are stored
images = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\Images\\'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')
# Get the length of the list
len(img)

31783

In [9]:
# import glob
# import os
# from sklearn.model_selection import train_test_split

# # Update the path to your Flickr30k Images folder
# images_folder = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\Images\\'

# # Create a list of all image names in the directory
# img = glob.glob(images_folder + '*.jpg')
# num_images = len(img)

# # Update the path to your captions file
# captions_file = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\captions.txt'

# # Load captions from the captions file
# with open(captions_file, 'r', encoding='utf-8') as file:
#     captions_data = file.readlines()

# # Assuming your captions file format is one caption per line
# captions_data = [line.strip() for line in captions_data]

# # Check if the number of images and captions match
# if num_images * 5 != len(captions_data):
#     print(f"Error: Found {num_images} images and {len(captions_data)//5} sets of 5 captions. Make sure they match.")
# else:
#     print(f"Number of images and sets of 5 captions match: {num_images}")

# # Split the data into training and testing sets
# train_img, test_img, train_captions, test_captions = train_test_split(img, captions_data, test_size=0.2, random_state=42)

# # Print the number of samples in each set
# print(f"Number of training samples: {len(train_img)}")
# print(f"Number of testing samples: {len(test_img)}")


In [14]:
import os
from sklearn.model_selection import train_test_split
from collections import defaultdict

# Update the path to your captions file
captions_file = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\captions.txt'

# Check if train and test captions files exist
train_captions_file = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\train_captions.txt'
test_captions_file = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\test_captions.txt'

# Number of captions to keep for each image ID
num_captions_per_image = 5

if os.path.exists(train_captions_file) and os.path.exists(test_captions_file):
    # Load train and test captions from existing files
    with open(train_captions_file, 'r', encoding='utf-8') as train_file:
        train_captions_data = train_file.readlines()

    with open(test_captions_file, 'r', encoding='utf-8') as test_file:
        test_captions_data = test_file.readlines()

    # Read captions from the captions file
    with open(captions_file, 'r', encoding='utf-8') as file:
        captions_data = file.readlines()

    # Split the data into image IDs and captions
    image_ids, captions = zip(*(line.strip().split(',', 1) for line in captions_data))

    # Split the data into training and testing sets
    train_ids, test_ids, train_captions, test_captions = train_test_split(image_ids, captions, test_size=0.2, random_state=42)

    # Create a defaultdict to store the captions for each image ID
    captions_mapping = defaultdict(list)

    # Populate the dictionary with all available captions
    for img_id, caption in zip(train_ids, train_captions):
        captions_mapping[img_id].append(caption)

    # Filter each image ID to have exactly num_captions_per_image captions for training data
    filtered_train_data = []
    for img_id, captions_list in captions_mapping.items():
        filtered_captions = captions_list[:num_captions_per_image]
        num_missing_captions = num_captions_per_image - len(filtered_captions)
        if num_missing_captions > 0:
            # If there are missing captions, duplicate existing captions to reach num_captions_per_image
            filtered_captions.extend(filtered_captions[:num_missing_captions])
        filtered_train_data.extend([(img_id, caption) for caption in filtered_captions])

    # Use the same defaultdict for the testing set
    captions_mapping_test = defaultdict(list)
    for img_id, caption in zip(test_ids, test_captions):
        captions_mapping_test[img_id].append(caption)
    
    # Filter each image ID in the test set to have exactly num_captions_per_image unique captions
    filtered_test_data = []
    for img_id, captions_list in captions_mapping_test.items():
        unique_captions = list(set(captions_list))
        num_unique_captions = len(unique_captions)
    
        # Repeat captions if there are fewer than num_captions_per_image
        for _ in range(num_captions_per_image):
            filtered_test_data.append((img_id, unique_captions[_ % num_unique_captions]))
    
    # Sort the testing set by image ID and caption for consistency
    sorted_test_data = sorted(filtered_test_data, key=lambda x: (x[0], x[1]))
    
    # Write sorted testing captions to a file
    with open(test_captions_file, 'w', encoding='utf-8') as test_file:
        for img_id, caption in sorted_test_data:
            test_file.write(f"{img_id}, {caption}\n")

    # Print the number of samples in each set
    print(f"Number of training samples: {len(filtered_train_data)}")
    print(f"Number of testing samples: {len(sorted_test_data)}")

Number of training samples: 156595
Number of testing samples: 107240


In [10]:
train_path = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\train_captions.txt'

# Load train image IDs from the train captions file
train_images = [line.split(",", 1)[0] for line in open(train_path, 'r', encoding='utf-8').read().split("\n")]
train_img = []

for im in img:
    extracted_id = im[len(images):]
    if extracted_id in train_images:
        train_img.append(im)

print(f"Number of training images: {len(train_img)}")

Number of training images: 31774


In [11]:
#The below code was used to check why the above extraction of number of training images is 0
# Print a sample of image IDs extracted from file paths
sample_extracted_ids = [im[len(images):] for im in img[:10]]
print("Sample of Extracted Image IDs:")
print(sample_extracted_ids)

# Print a sample of image IDs from the captions file
sample_train_image_ids = train_images[:10]
print("Sample of Train Image IDs:")
print(sample_train_image_ids)

Sample of Extracted Image IDs:
['1000092795.jpg', '10002456.jpg', '1000268201.jpg', '1000344755.jpg', '1000366164.jpg', '1000523639.jpg', '1000919630.jpg', '10010052.jpg', '1001465944.jpg', '1001545525.jpg']
Sample of Train Image IDs:
['3773310720.jpg', '3773310720.jpg', '3773310720.jpg', '3773310720.jpg', '3773310720.jpg', '4524418308.jpg', '4524418308.jpg', '4524418308.jpg', '4524418308.jpg', '4524418308.jpg']


In [12]:
# Update the path to your test captions file
test_path = 'C:\\Users\\Aditya\\Desktop\\Fl_30_Dataset\\test_captions.txt'

# Load test image IDs from the test captions file
test_images = [line.split(",", 1)[0] for line in open(test_path, 'r', encoding='utf-8').read().split("\n")]
test_img = []

for im in img:
    extracted_id = im[len(images):]
    if extracted_id in test_images:
        test_img.append(im)

print(f"Number of testing images: {len(test_img)}")

Number of testing images: 21448


In [13]:
#load descriptions of train and test set separately
def load_clean_descriptions(des, dataset):
    dataset_des = dict()
    for key, des_list in des.items():
        if key+'.jpg' in dataset:
            if key not in dataset_des:
                dataset_des[key] = list()
            for line in des_list:
                desc = 'startseq ' + line + ' endseq'
                dataset_des[key].append(desc)
    return dataset_des

# Load train and test descriptions
train_descriptions = load_clean_descriptions(descriptions, train_images)
test_descriptions = load_clean_descriptions(descriptions, test_images)

print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))

Descriptions: train=31774, test=21448


In [14]:
print('Train')
train_descriptions['3773310720']

Train


['startseq man in sleeveless brown shirt and khaki shorts sews on piece of cloth covering ramshackle shade structure endseq',
 'startseq man in shorts standing by the water endseq',
 'startseq man is working on his wooden boat endseq',
 'startseq man in brown building raft endseq',
 'startseq man making canvas shade endseq']

In [15]:
print('Test')
test_descriptions['1000092795']

Test


['startseq two young guys with shaggy hair look at their hands while hanging out in the yard endseq',
 'startseq two young white males are outside near many bushes endseq',
 'startseq two men in green shirts are standing in yard endseq',
 'startseq man in blue shirt standing in garden endseq',
 'startseq two friends enjoy time spent together endseq']

In [16]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from keras.preprocessing.image import load_img, img_to_array
def preprocess_img(img_path):
    #inception v3 excepts img in 299*299
    img = load_img(img_path, target_size = (299, 299))
    x = img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    return x

In [17]:
base_model = InceptionV3(weights = 'imagenet')
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [18]:
model = Model(base_model.input, base_model.layers[-2].output)

In [19]:
#function to encode an image into a vector using inception v3
def encode(image):
    image = preprocess_img(image)
    vec = model.predict(image)
    vec = np.reshape(vec, (vec.shape[1]))
    return vec

In [41]:
# #run the encode function on all train images
# start = time.time()
# encoding_train = {}
# for img in train_img:
#     encoding_train[img[len(images):]] = encode(img)
# print("Time Taken is: " + str(time.time() - start))

1/1 [==============================] - 0s 235ms/step
Time Taken is: 11389.33042383194


In [ ]:
# Run the encode function on all train images

import pickle

start = time.time()
encoding_train = {}

for img in train_img:
    encoding_train[img[len(images):]] = encode(img)

# Load the encoding dictionary from the file
save_path = 'encoding_train_final.pkl'
with open(save_path, 'rb') as file:
    encoding_train = pickle.load(file)

print("Final Time Taken is: " + str(time.time() - start))

# Clear the memory
del encoding_train

# Optional: Garbage collect to free up additional memory
import gc
gc.collect()


1/1 [==============================] - ETA: 0s

In [5]:
#Encode all the test images
start = time.time()
encoding_test = {}
for img in test_img:
    encoding_test[img[len(images):]] = encode(img)
print("Time taken is: " + str(time.time() - start))

NameError: name 'test_img' is not defined